<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Quiz%20Practice/Tool_Use_Behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_export_api_key, trace
from openai import AsyncOpenAI
from google.colab  import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
set_tracing_export_api_key(OPENAI_API_KEY)

Client = AsyncOpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash-lite",
    openai_client= Client
)                                                                      #Setting up the environment

#Synchronus & Asynchronus Behaviour

In [ ]:
import asyncio
from agents import function_tool

@function_tool
def get_weather(location: str) -> str:          #Async Runner & Agent with sync function as tool
  return f"The weather in {location} is Sunny"

async def main():
  agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
  )

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karachi?")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny.


In [ ]:
import asyncio
from agents import function_tool

@function_tool
def get_weather(location: str) -> str:
  return f"The weather in {location} is Sunny"

agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
 )

async def main():

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karachi?")
    print(res.final_output)           #Async Runner  with synchronus Agent & function  as a tool

if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny.


In [ ]:
import asyncio
from agents import function_tool

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:          #Async Runner & Agent & function as tool
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
  )

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karachi?")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny.


In [ ]:
import asyncio
from agents import function_tool

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:          #Async  Agent & function as tool with sync runner
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
  )

if __name__ == "__main__":
  asyncio.run(main())

with trace("Tool Use"):
 res =  Runner.run_sync(agent,"What's weather like in Karachi?")
 print(res)

RunResult:
- Last agent: Agent(name="Assistant", ...)
- Final output (str):
    The weather in Karanchi is Sunny.
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


#Tool Use Behaviour

In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather],
      model_settings = ModelSettings(temperature=1, tool_choice="required"),
  )                                              # tool_choice = auto is by default and llm will make sure if it needs tool
                                                 # tool_choice = required will make sure tool is called no matter the prompt
  with trace("Tool Use"):           # tool_choice = none or empty will make sure tool is never called
    res = await Runner.run(agent,"Hey there!")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

A sunny day greets,
New York's clear skies, a bright start,
Nature's gentle smile.


In [ ]:
import asyncio
from agents import function_tool

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather],
      tool_use_behavior= "stop_on_first_tool",
  )

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karachi?")
    print(res.final_output)

    #   In traces we find out that because of tool_use_behavior= "stop_on_first_tool" the
    #   agent loop stops and original output of the tool is saved as final_output.
    #   by default it is "run_llm_again"
if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny


In [ ]:
import asyncio
from agents import function_tool
from agents.agent import StopAtTools

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  @function_tool
  async def  get_news(location: str) -> str:
   return  f"Only Good news are coming from {location}"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather, get_news],
      tool_use_behavior= StopAtTools(stop_at_tool_names=["get_weather"])
  )
#   Stop at Tool use but for specific tool from multiple tools

  with trace("Tool Use"):
    res = await Runner.run(agent,"What are news like in Karachi & how's weather like?")
    print(res.final_output)
#  LLM calls both tools but since it's instructed to stop at "get_weather", it stops and returns the
#  "get_weather" output to final_output
if __name__ == "__main__":
  asyncio.run(main())

The weather in Karachi is Sunny


In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather],
      model_settings = ModelSettings(temperature=1, tool_choice="required"),
      reset_tool_choice=True, # this is prefered now and tool choice required is now not implemented

  )                                              # tool_choice = auto is by default and llm will make sure if it needs tool
                                                 # tool_choice = required will make sure tool is called no matter the prompt
  with trace("Tool Use"):           # tool_choice = "none" /None or empty will make sure tool is never called
    res = await Runner.run(agent,"Hey there!")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

Good day to you!
How may I help you today?
Tell me your desires.


In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather],
  )
  #  Max turns exceeded error will be thrown
  with trace("Tool Use"):
    res = await Runner.run(agent,"What's the weather like in Karachi?",max_turns=1)
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

MaxTurnsExceeded: Max turns (1) exceeded

In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather],
      model_settings = ModelSettings(temperature=1, tool_choice="required"),
      reset_tool_choice=False, # this is prefered now and tool choice reset will bring us into tool calling loop
  )        # if max_turns would not have been applied it will continue calling tools and then llm until api rate limit or perhaps default max_turns = 10
                                         #  Max turns exceeded error will be thrown
  with trace("Tool Use"):
    res = await Runner.run(agent,"Hi",max_turns=3)
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

MaxTurnsExceeded: Max turns (3) exceeded

In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather],
      model_settings = ModelSettings(temperature=1, tool_choice=".45.,/j["),
      reset_tool_choice=False,

  )                                              # tool_choice = auto/None is by default and llm will make sure if it needs tool
                                                 # tool_choice = required will make sure tool is called no matter the prompt
  with trace("Tool Use"):           # tool_choice = "none" or empty or any string value will make sure tool is never called
    res = await Runner.run(agent,"What's the weather in Karachi")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

Karachi's weather
Sunny skies, warm, gentle breeze
A pleasant day calls


#Parallel Tool Call

In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:
   return  f"The weather in {location} is Sunny"

  @function_tool
  async def  get_news(location: str) -> str:
   return  f"Positive News are coming from {location}"

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather,get_news],
      model_settings = ModelSettings(parallel_tool_calls=False),
  )
                                       # Ok since the whole nb is Async the parallel_tool_calls=False is not stopping parallel tool call
  with trace("Tool Use"):            # we will try again in next cell with synchronus stuff
    res = await Runner.run(agent,"What's the weather in Karachi and what kind of news are coming out of there?")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

Sunny skies appear,
Good news from Karachi comes,
Hope fills the warm air.



In [ ]:
import asyncio
from agents import function_tool, model_settings, ModelSettings

@function_tool
def  get_weather(location: str) -> str:
 return  f"The weather in {location} is Sunny"

@function_tool
def  get_news(location: str) -> str:
 return  f"Positive News are coming from {location}"

async def main():

  agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather,get_news],
      model_settings = ModelSettings(parallel_tool_calls=False),
  )
                                       # Ok now since the tools are sync and rest of nb is Async the parallel_tool_calls=False is not stopping parallel tool call
  with trace("Tool Use"):           # try again next cell
    res = await Runner.run(agent,"What's the weather in Karachi and what kind of news are coming out of there?")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

Sun shines in Karachi,
Good news brings a happy day,
Peace fills the warm air.



In [ ]:
from agents import function_tool, model_settings, ModelSettings

@function_tool
def  get_weather(location: str) -> str:
 return  f"The weather in {location} is Sunny"

@function_tool
def  get_news(location: str) -> str:
 return  f"Positive News are coming from {location}"

agent = Agent(
      name = "Assistant",
      instructions = "respond in haiku",
      model = model,
      tools = [get_weather,get_news],
      model_settings = ModelSettings(parallel_tool_calls=False),
  )
                                       # Ok now since whole nb is Async the parallel_tool_calls=False is still not stopping parallel tool call
with trace("Tool Use"):
    res =  Runner.run_sync(agent,"What's the weather in Karachi and what kind of news are coming out of there?")
    print(res.final_output)

The weather is sunny
Good news is coming from there
Karachi is bright


#Dynamic Instructions - Context Engineering

In [ ]:
def dynamic_instructions(context, agent) -> str:
  print(f"context: {context}")
  print(f"\nagent: {agent}\n")
  return f"Respond in haiku"
                                # without typing
agent = Agent(
      name = "Assistant",
      instructions = dynamic_instructions,
      model = model,
  )

with trace("Dynamic Instructions"):
    res =  Runner.run_sync(agent,"Yo 🥀 ")
    print(res.final_output)

context: RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))

agent: Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=<function dynamic_instructions at 0x785ff0ef11c0>, prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x785ffd312210>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

C

In [ ]:
from agents import RunContextWrapper

def dynamic_instructions(ctx: RunContextWrapper, starting_agent: Agent) -> str:
  print(f"context: {ctx}")
  print(f"\ncontext-context: {ctx.context}")
  print(f"\nagent: {starting_agent}")
  print(f"\nagent-model: {starting_agent.model}\n")
  return f"Respond in haiku"
                                                       # with typing
agent = Agent(
      name = "Assistant",
      instructions = dynamic_instructions,
      model = model,
  )

with trace("Dynamic Instructions"):
    res =  Runner.run_sync(agent,"Yo 🥀 ")
    print(res.final_output)

context: RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))

context-context: None

agent: Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=<function dynamic_instructions at 0x785ff0ef1da0>, prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x785ffd312210>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', res

In [ ]:
def dynamic_instructions(agent, context) -> str:
  print(f"context: {context}") # agent is stored as context
  print(f"\nagent: {agent}\n")# context is stored as agent
  return f"Respond in haiku"
                                # without typing parameter swap
agent = Agent(
      name = "Assistant",
      instructions = dynamic_instructions,
      model = model,
  )

with trace("Dynamic Instructions"):
    res =  Runner.run_sync(agent,"Yo 🥀 ")
    print(res.final_output)

context: Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=<function dynamic_instructions at 0x785ff0ef19e0>, prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x785ffd312210>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

agent: RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))

P

In [ ]:
from agents import RunContextWrapper

def dynamic_instructions(starting_agent: Agent, ctx: RunContextWrapper) -> str:
  print(f"context: {ctx}")
  print(f"\ncontext-context: {ctx.context}")
  print(f"\nagent: {starting_agent}")
  print(f"\nagent-model: {starting_agent.model}\n")
  return f"Respond in haiku"
                                                       # with typing parameter swap ctx doesnt have context property the values swaped that's why error
agent = Agent(
      name = "Assistant",
      instructions = dynamic_instructions,
      model = model,
  )

with trace("Dynamic Instructions"):
    res =  Runner.run_sync(agent,"Yo 🥀 ")
    print(res.final_output)

context: Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=<function dynamic_instructions at 0x785ff0ef1120>, prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x785ffd312210>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)


AttributeError: 'Agent' object has no attribute 'context'

In [ ]:
from agents import RunContextWrapper

def dynamic_instructions(ctx: RunContextWrapper, starting_agent: Agent) -> str:
  print(f"context: {ctx}")
  print(f"\ncontext-context: {ctx.context}")
  print(f"\nagent: {starting_agent}")
  print(f"\nagent-instructions: {starting_agent.instructions}\n")
  return f"Respond in haiku"
                                                       # context a python object passed in runner
agent = Agent(
      name = "Assistant",
      instructions = dynamic_instructions,
      model = model,
  )

with trace("Dynamic Instructions"):
    res =  Runner.run_sync(agent,"Yo 🥀 ",context="Ali")
    print(res.final_output)

context: RunContextWrapper(context='Ali', usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))

context-context: Ali

agent: Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=<function dynamic_instructions at 0x785ff0f43b00>, prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x785ffd312210>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', res